Imports

In [10]:
import pandas as pd
import random
import numpy as np
import warnings

Global functions

In [3]:
def get_accident_df(subset="standard_small", n_rows=10000, is_refined=False):
    warnings.warn("This function is outdated and should not be used.")
    if is_refined:
        df = get_accident_df(subset=subset, n_rows=n_rows, is_refined=False)
        return refine_df(df)

    if subset == "standard_small":
        return pd.read_csv("DataSubsetRaw.csv")
    if subset == "new":
        n = 3513617 #number of rows in the dataset
        s = n_rows #number of rows to sample
        skip = sorted(random.sample(range(1, n+1), n-s)) #rows to be skipped
        return pd.read_csv("US_Accidents_June20.csv", skiprows=skip)
    if subset == "all":
        return pd.read_csv("US_Accidents_June20.csv")

Internal functions

In [12]:
def refine_df(df):
    #Modify the df in various ways
    return df

Testing

In [4]:
df = pd.read_csv("DataSubsetRaw.csv")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 50 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             10000 non-null  int64  
 1   ID                     10000 non-null  object 
 2   Source                 10000 non-null  object 
 3   TMC                    7008 non-null   float64
 4   Severity               10000 non-null  int64  
 5   Start_Time             10000 non-null  object 
 6   End_Time               10000 non-null  object 
 7   Start_Lat              10000 non-null  float64
 8   Start_Lng              10000 non-null  float64
 9   End_Lat                2992 non-null   float64
 10  End_Lng                2992 non-null   float64
 11  Distance(mi)           10000 non-null  float64
 12  Description            10000 non-null  object 
 13  Number                 3540 non-null   float64
 14  Street                 10000 non-null  object 
 15  Sid

In [15]:
df["Pressure(in)"].head()

0    29.84
1    29.31
2    30.04
3    30.45
4    30.12
Name: Pressure(in), dtype: float64

In [ ]:
def convert_to_metric(df):
    df["Distance(km)"] = df["Distance(mi)"] * 1.60934
    df["Temperature(C)"] = (df["Temperature(F)"] - 32) * 5 / 9
    df["Wind_Chill(C)"] = (df["Wind_Chill(F)"] - 32) * 5 / 9
    df["Pressure(bar)"] = df["Pressure(in)"] / 29.53
    return df